In [376]:
import pandas as pd
import numpy as np

In [377]:
car = pd.read_csv('quikr_car.csv')

In [378]:
car.head()

,name,company,year,Price,kms_driven,fuel_type
0,Hyundai Santro Xing XO eRLX Euro III,Hyundai,2007,"80,000","45,000 kms",Petrol
1,Mahindra Jeep CL550 MDI,Mahindra,2006,"4,25,000",40 kms,Diesel
2,Maruti Suzuki Alto 800 Vxi,Maruti,2018,Ask For Price,"22,000 kms",Petrol
3,Hyundai Grand i10 Magna 1.2 Kappa VTVT,Hyundai,2014,"3,25,000","28,000 kms",Petrol
4,Ford EcoSport Titanium 1.5L TDCi,Ford,2014,"5,75,000","36,000 kms",Diesel


In [379]:
car.shape

(892, 6)

In [380]:
car.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 892 entries, 0 to 891
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        892 non-null    object
 1   company     892 non-null    object
 2   year        892 non-null    object
 3   Price       892 non-null    object
 4   kms_driven  840 non-null    object
 5   fuel_type   837 non-null    object
dtypes: object(6)
memory usage: 41.9+ KB


In [381]:
# car['year'].unique()
# car['Price'].unique()


In [382]:
## Quality of data
# names are pretty inconsistent
# names have company names attached to it
# some names are spam like 'Maruti Ertiga showroom condition with' and 'Well mentained Tata Sumo'
# company: many of the names are not of any company like 'Used', 'URJENT', and so on.
# year has many non-year values
# year is in object. Change to integer
# Price has Ask for Price
# Price has commas in its prices and is in object
# kms_driven has object values with kms at last.
# It has nan values and two rows have 'Petrol' in them
# fuel_type has nan values

In [383]:
# Cleaning

In [384]:
backup = car.copy()

In [385]:
# Convert the year to integer
car = car[car['year'].str.isnumeric()]

In [386]:
# Convert the data type of year to integer
car['year'] = car['year'].astype('int32')

In [387]:
car = car[car['Price']!= 'Ask For Price']

In [388]:
car['Price']=car['Price'].str.replace(',','').astype('int32')

In [389]:
car.info()

<class 'pandas.core.frame.DataFrame'>
Index: 819 entries, 0 to 891
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        819 non-null    object
 1   company     819 non-null    object
 2   year        819 non-null    int32 
 3   Price       819 non-null    int32 
 4   kms_driven  819 non-null    object
 5   fuel_type   816 non-null    object
dtypes: int32(2), object(4)
memory usage: 38.4+ KB


In [390]:
car['kms_driven'] = car['kms_driven'].str.split().str.get(0).str.replace(',','')

In [391]:
car=car[car['kms_driven'].str.isnumeric()]

In [392]:
car['kms_driven'] = car['kms_driven'].astype('int32')

In [393]:
car=car[~car['fuel_type'].isna()]

In [394]:
car['name'] = car['name'].str.split(' ').str.slice(0, 3).str.join(' ')

In [395]:
car = car.reset_index(drop=True)

In [396]:
car = car[car['Price'] < 6e6].reset_index(drop=True)

In [397]:
car.to_csv('cleaned_car.csv')

In [398]:
#Model

In [399]:
X = car.drop(columns=['Price'])
y = car['Price']

In [400]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [401]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [402]:
from sklearn.compose import make_column_transformer


ohe = OneHotEncoder(handle_unknown='ignore')
ohe.fit(X_train[['name', 'company', 'fuel_type']])

column_trans = make_column_transformer((OneHotEncoder(categories=ohe.categories_, handle_unknown='ignore'), ['name', 'company', 'fuel_type']), remainder='passthrough')


In [403]:
lr = LinearRegression()

In [404]:
from sklearn.pipeline import make_pipeline


pipe=make_pipeline(column_trans,lr)
pipe.fit(X_train, y_train)

,steps,"[('columntransformer', ...), ('linearregression', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('onehotencoder', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [405]:
y_pred=pipe.predict(X_test)


In [406]:

r2_score(y_test,y_pred)

0.6136779992235872

In [407]:
scores=[]
for i in range(1000):
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.1,random_state=i)
    lr=LinearRegression()
    pipe=make_pipeline(column_trans,lr)
    pipe.fit(X_train,y_train)
    y_pred=pipe.predict(X_test)
    scores.append(r2_score(y_test,y_pred))

In [409]:
np.argmax(scores)

np.int64(302)

In [410]:
scores[np.argmax(scores)]

0.9006778658781721

In [411]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.1,random_state=302)
lr=LinearRegression()
pipe=make_pipeline(column_trans,lr)
pipe.fit(X_train,y_train)
y_pred=pipe.predict(X_test)
scores.append(r2_score(y_test,y_pred))
r2_score(y_test,y_pred)

0.9006778658781721

In [412]:
import pickle